In [ ]:
import urllib.request
url = "https://github.com/open-mmlab/mmdetection/blob/main/configs/solo "
urllib.request.urlretrieve(url, "checkpoints/solov2_r50_fpn_1x_coco.pth")


In [1]:
import mmdet.apis
print(dir(mmdet.apis))  # This will list all available functions


C:\Users\AbayM\anaconda3\Lib\site-packages\mmcv\__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


['__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__path__', '__spec__', 'async_inference_detector', 'get_root_logger', 'inference', 'inference_detector', 'init_detector', 'init_random_seed', 'multi_gpu_test', 'set_random_seed', 'show_result_pyplot', 'single_gpu_test', 'test', 'train', 'train_detector']


In [11]:
print(cfg.data)


{'imgs_per_gpu': 2, 'workers_per_gpu': 2, 'train': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_train2017.json', 'img_prefix': 'data/coco/train2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'LoadAnnotations', 'with_bbox': True, 'with_mask': True}, {'type': 'Resize', 'img_scale': [(1333, 800), (1333, 768), (1333, 736), (1333, 704), (1333, 672), (1333, 640)], 'multiscale_mode': 'value', 'keep_ratio': True}, {'type': 'RandomFlip', 'flip_ratio': 0.5}, {'type': 'Normalize', 'mean': [123.675, 116.28, 103.53], 'std': [58.395, 57.12, 57.375], 'to_rgb': True}, {'type': 'Pad', 'size_divisor': 32}, {'type': 'DefaultFormatBundle'}, {'type': 'Collect', 'keys': ['img', 'gt_bboxes', 'gt_labels', 'gt_masks']}]}, 'val': {'type': 'CocoDataset', 'ann_file': 'data/coco/annotations/instances_val2017.json', 'img_prefix': 'data/coco/val2017/', 'pipeline': [{'type': 'LoadImageFromFile'}, {'type': 'MultiScaleFlipAug', 'img_scale': (1333, 800), 'flip': False, 'transforms': 

In [43]:
import os
import mmcv
from mmdet.datasets import build_dataset
from mmdet.apis import train_detector
from mmdet.models import build_detector
from mmcv import Config


# Путь к директории с данными
data_path = r'C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco'

# Загружаем конфигурационный файл для SOLOv2
config_file = r"C:\Users\AbayM\Downloads\solo\SOLO\configs\solo\solo_r101_fpn_8gpu_3x.py" # Путь к конфигу SOLOv2, пример с backbone ResNet50
cfg = Config.fromfile(config_file)

# Correctly update the paths for your datasets

cfg.data.train.img_prefix = r"C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco\train"
cfg.data.val.img_prefix = r"C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco\val"
cfg.data.test.img_prefix = r"C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco\test"

# Update the annotation files
cfg.data.train.ann_file = r"C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco\annotations\instances_train2017.json"
cfg.data.val.ann_file = r"C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco\annotations\instances_val2017.json"
cfg.data.test.ann_file = r"C:\Users\AbayM\Downloads\solo\SpaceVela 3D-CV 1 class fail.v16i.coco\annotations\instances_val2017.json"

# If runner configuration does not exist in the config, define it manually
cfg.runner = dict(type='EpochBasedRunner', max_epochs=12)  # Set to Epoch-based runner and specify max_epochs


# Настройка пути для сохранения модели
cfg.work_dir = './work_dirs/solo'

# Стартуем обучение
cfg.gpus = 1  # Используем одну видеокарту
cfg.runner.max_epochs = 12  # Устанавливаем количество эпох

# Строим модель
model = build_detector(cfg.model)
datasets = [build_dataset(cfg.data.train)]
train_detector(model, datasets, cfg, distributed=False, validate=True)


TypeError: SOLO: SOLOHead: SOLOHead.__init__() got an unexpected keyword argument 'seg_feat_channels'

In [17]:
from SOLO.mmdet.models import build_detector
from mmdet.apis import inference_detector, show_result_pyplot
from mmcv import Config
import mmcv

# Load the config and model checkpoint
cfg = Config.fromfile('C:/Users/AbayM/Downloads/solo/mmdetection/configs/solov2/solov2_r50_fpn_1x_coco.py')
cfg.load_from = 'C:/Users/AbayM/Downloads/solo/work_dirs/solov2_train/latest.pth'  # Adjust to the path where your trained model is saved

# Build the model for inference
model = build_detector(cfg.model, test_cfg=cfg.test_cfg)
checkpoint = mmcv.runner.load_checkpoint(model, cfg.load_from)

# Inference on a test image
img = 'C:/path/to/test/image.jpg'  # Replace with the actual test image path
result = inference_detector(model, img)

# Show results
show_result_pyplot(model, img, result)


AssertionError: MMCV==2.2.0 is used but incompatible. Please install mmcv>=2.0.0rc4, <2.2.0.

In [47]:
# Гиперпараметры обучения
num_classes = 1  # У нас всего 1 класс "fail"
batch_size = 2
learning_rate = 0.01
num_epochs = 50
seed = 42  # Фиксируем random_state для воспроизводимости


In [49]:
_base_ = './solov2_r50_fpn_1x_coco.py'

dataset_type = 'CocoDataset'
data_root = 'C:/Users/AbayM/Downloads/solo/'

classes = ('fail',)

data = dict(
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'train/_annotations.coco.json',
        img_prefix=data_root + 'train/',
        classes=classes
    ),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'valid/_annotations.coco.json',
        img_prefix=data_root + 'valid/',
        classes=classes
    ),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'test/_annotations.coco.json',
        img_prefix=data_root + 'test/',
        classes=classes
    )
)

model = dict(
    roi_head=dict(
        bbox_head=dict(num_classes=1),
        mask_head=dict(num_classes=1)
    )
)

random_seed = 42  # Фиксируем random_state


In [53]:
_base_ = './solov2_r50_fpn_1x_coco.py'

# Dataset settings
dataset_type = 'CocoDataset'
data_root = 'C:/Users/AbayM/Downloads/solo/'

classes = ('fail',)

data = dict(
    train=dict(
        type=dataset_type,
        ann_file=data_root + 'train/_annotations.coco.json',
        img_prefix=data_root + 'train/',
        classes=classes
    ),
    val=dict(
        type=dataset_type,
        ann_file=data_root + 'valid/_annotations.coco.json',
        img_prefix=data_root + 'valid/',
        classes=classes
    ),
    test=dict(
        type=dataset_type,
        ann_file=data_root + 'test/_annotations.coco.json',
        img_prefix=data_root + 'test/',
        classes=classes
    )
)

# Model settings
model = dict(
    type='SOLO',
    pretrained='torchvision://resnet101',
    backbone=dict(
        type='ResNet',
        depth=101,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet101')
    ),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        start_level=0,
        num_outs=5
    ),
    bbox_head=dict(
        type='SOLOHead',
        num_classes=1,  # Adjusted for a single class
        in_channels=256,
        stacked_convs=7,
        strides=[8, 8, 16, 32, 32],
        scale_ranges=((1, 96), (48, 192), (96, 384), (192, 768), (384, 2048)),
        sigma=0.2,
        num_grids=[40, 36, 24, 16, 12],
        cate_down_pos=0,
        with_deform=False,
        loss_ins=dict(
            type='DiceLoss',
            use_sigmoid=True,
            loss_weight=3.0
        ),
        loss_cate=dict(
            type='FocalLoss',
            use_sigmoid=True,
            gamma=2.0,
            alpha=0.25,
            loss_weight=1.0
        ),
    )
)

# Optimizer and Learning Rate Schedule
optimizer = dict(type='SGD', lr=0.01, momentum=0.9, weight_decay=0.0001)
optimizer_config = dict(grad_clip=dict(max_norm=35, norm_type=2))

lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=0.001,
    step=[40, 80]
)

# Training settings
total_epochs = 100
checkpoint_config = dict(interval=10)
evaluation = dict(interval=1, metric=['bbox', 'segm'])
log_config = dict(
    interval=50,
    hooks=[dict(type='TextLoggerHook')]
)

dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = None
resume_from = None
workflow = [('train', 1)]

random_seed = 42

# Evaluation and Visualization
work_dir = './work_dirs/solo_custom/'

visualization = dict(
    type='TensorboardLoggerHook',
    log_dir=work_dir + '/logs'
)

def visualize_results(result_file, dataset):
    from mmdet.apis import init_detector, inference_detector, show_result_pyplot
    import mmcv
    
    config_file = work_dir + 'latest.pth'
    model = init_detector(config_file, result_file, device='cuda:0')
    img = dataset + '/test/sample.jpg'  # Change to any test image
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result, score_thr=0.3)


